In [1]:
import pandas as pd

# 为每一列定义数据类型
renting_dtype = {
    '房源编号': 'string',
    '出租方式': 'category',
    '街道': 'object',
    '小区': 'object',
    '租金': 'float64',
    '卧室数量': 'int64',
    '客厅数量': 'int64',
    '面积': 'float64',
    '朝向': 'category',
    '楼层': 'category',
    '最高楼层': 'int64',
    '装修情况': 'object',
    '关注人数': 'int64',
    '近30天带看': 'int64',
    # '发布时间': 'datetime64',  # 需要在读取时，使用parse_dates=['发布时间']
    '所在经度': 'float64', '所在纬度': 'float64',
    '最近的地铁站经度': 'float64', '最近的地铁站纬度': 'float64',
    '地铁直线距离': 'float64', '地铁步行距离': 'float64', '地铁步行时长': 'float64',
    '最近的公交站经度': 'float64', '最近的公交站纬度': 'float64',
    '公交直线距离': 'float64', '公交步行距离': 'float64', '公交步行时长': 'float64',
    '1km内餐厅': 'int64',
    '1km内大学或大专': 'int64',
    '1km内办公室': 'int64',
    '1km内医院': 'int64',
    '1km内健身房': 'int64',
    '房源亮点': 'string',
    '户型介绍': 'string',
    '交通出行': 'string',
    '周边配套': 'string',
    '小区信息': 'string',
    '洗衣机': 'boolean', '冰箱': 'boolean', '电视': 'boolean',
    '空调': 'boolean', '热水器': 'boolean', '天然气': 'boolean',
    '暖气': 'boolean', '床': 'boolean', '网络': 'boolean',
    '衣柜': 'boolean',
    '交通': 'str',
    '生活': 'str',
    '品质': 'str',
    '医疗': 'str',
    '运动': 'str',
    '最低参考价': 'float64',
    '最高参考价': 'float64'
}

# 加载具有定义数据类型和指定标题行的CSV文件
data = pd.read_csv("./input/NLP后-全量数据.csv",
                   encoding="utf-8",
                   dtype=renting_dtype,
                   parse_dates=['发布时间'],
                   header=0)

# 显示DataFrame信息以确认数据类型
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26296 entries, 0 to 26295
Data columns (total 60 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   房源编号           26296 non-null  string        
 1   出租方式           26296 non-null  category      
 2   街道             26296 non-null  object        
 3   小区             26296 non-null  object        
 4   租金             26296 non-null  float64       
 5   卧室数量           26296 non-null  int64         
 6   客厅数量           26296 non-null  int64         
 7   面积             26296 non-null  float64       
 8   朝向             26296 non-null  category      
 9   楼层             26296 non-null  category      
 10  最高楼层           26296 non-null  int64         
 11  装修情况           26061 non-null  object        
 12  关注人数           26296 non-null  int64         
 13  近30天带看         26296 non-null  int64         
 14  发布时间           26296 non-null  datetime64[ns]
 15  所在经度           2629

## 数据清洗和格式化

对于房地产租赁数据，我们首先需要确保所有的数据格式都是正确的，并且进行适当的数据清洗。这包括数据类型转换，空值处理，去除特殊字符以及格式标准化等步骤。以下Python代码展示了如何对数据集进行这些基本的处理操作。


In [2]:
import numpy as np

# 定义其他类别的列列表
pois = ['便利店', '综合超市', '商场', '药店', '银行']

for poi in pois:
    # 将列转换为数值型，非数值转为NaN
    data[poi + '_numeric'] = pd.to_numeric(data[poi], errors='coerce')
    # 计算非NaN数值的中位数和第一四分位数（IQR1）
    median_value = data[poi + '_numeric'].median()
    IQR1 = data[poi + '_numeric'].quantile(0.25)
    # 替换'None'为0，“较少”为IQR1，NaN用中位数填充，然后转换为整型
    data[poi] = data[poi].replace('None', 0).replace(
        '较少', IQR1).fillna(median_value).astype(int)
    # 删除临时的数值列
    data.drop(poi + '_numeric', axis=1, inplace=True)

In [3]:
# 使用“毛坯”填充“装修情况”列的缺失值
data["装修情况"] = data["装修情况"].fillna("毛坯")

# 定义需要进行数字处理的列名列表
canteens = ['平价餐厅', '5星餐厅', '4星餐厅']

# 遍历每个数值列并应用清理步骤
for canteen in canteens:
    data[canteen] = pd.to_numeric(data[canteen], errors='coerce').fillna(0).astype(int)

# 通过汇总三个餐厅列来计算“附近餐厅数”
data['附近餐厅数'] = data['平价餐厅'] + data['5星餐厅'] + data['4星餐厅']

In [4]:
hospitals = ['三级医院数量', '二级医院数量']

for hospital in hospitals:
    # 将数据转换为数字类型，错误处理方式为将错误强制转换为NaN
    data[hospital] = pd.to_numeric(data[hospital], errors='coerce')

    # 计算中位数和IQR1，仅考虑非NaN值
    median_value = data[hospital].median(skipna=True)
    IQR1 = data[hospital].quantile(0.25, interpolation='midpoint')

    # 用计算出的统计量替换特定的占位符
    data[hospital] = data[hospital].replace(['不多', '较少'], IQR1)
    # 将'None'及类似值替换为0
    data[hospital] = data[hospital].replace(['不明', '不明确', 'None'], 0)

    # 用中位数填充剩余的NaN值
    data[hospital] = data[hospital].fillna(median_value)

    # 检查是否有剩余的非有限值并替换它们
    if not np.isfinite(data[hospital]).all():
        data[hospital] = data[hospital].replace(
            [np.inf, -np.inf], median_value)

    data[hospital] = data[hospital].astype(int)

data["最近医院距离（米）"] = data["最近医院距离（米）"].fillna('None')
data["（最近医院）是否是三甲医院"] = data["（最近医院）是否是三甲医院"].fillna(False)

# 将医院数量汇总到一个新列
data['附近医院数'] = data[hospitals].sum(axis=1)

print(data[['最近医院距离（米）', '（最近医院）是否是三甲医院', '附近医院数'] + hospitals].head())

  最近医院距离（米） （最近医院）是否是三甲医院  附近医院数  三级医院数量  二级医院数量
0      None         False      2       1       1
1      None         False      2       1       1
2      None         False      2       1       1
3       560         FALSE      2       1       1
4      None         False      2       1       1


In [5]:
# 定义特征列列表
appliances = [
    '洗衣机', '冰箱', '电视', '空调',
    '热水器', '天然气', '暖气', '床', '网络', '衣柜'
]

# 遍历每个特征列计算平均值并应用75%规则
for appliance in appliances:
    data[appliance] = data[appliance].replace({'None': None, '': None})  # 如有必要，规范化None表示

    # 计算除NaN之外的True值的百分比
    percentage_true = data[appliance].dropna().mean()

    # 应用75%规则
    if percentage_true >= 0.75:
        fill_value = True
    else:
        fill_value = False

    # 根据规则填充缺失值
    data[appliance] = data[appliance].fillna(fill_value)

# print(data[appliances].head())

In [6]:
def contains_chinese(text):
    """
    检查给定文本中是否包含中文字符。

    参数:
        text (str): 需要检查的文本字符串。

    返回:
        bool: 如果文本包含至少一个中文字符，则返回True，否则返回False。
    """
    if pd.isna(text):
        return False  # 将NaN和None视为False
    for character in text:
        if '\u4e00' <= character <= '\u9fff':
            return True
    return False


data['停车'] = data['停车'].apply(lambda x: contains_chinese(
    x) if x not in [None, '', 'None'] else False)

print(data[['停车']].head())

      停车
0  False
1  False
2  False
3  False
4   True


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26296 entries, 0 to 26295
Data columns (total 62 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   房源编号           26296 non-null  string        
 1   出租方式           26296 non-null  category      
 2   街道             26296 non-null  object        
 3   小区             26296 non-null  object        
 4   租金             26296 non-null  float64       
 5   卧室数量           26296 non-null  int64         
 6   客厅数量           26296 non-null  int64         
 7   面积             26296 non-null  float64       
 8   朝向             26296 non-null  category      
 9   楼层             26296 non-null  category      
 10  最高楼层           26296 non-null  int64         
 11  装修情况           26296 non-null  object        
 12  关注人数           26296 non-null  int64         
 13  近30天带看         26296 non-null  int64         
 14  发布时间           26296 non-null  datetime64[ns]
 15  所在经度           2629

In [8]:
# 计算租金的低和高分位数（5%和95%分位数）
rent_low = data['租金'].quantile(0.05)
rent_high = data['租金'].quantile(0.95)

# 筛选出租金异常值
rental_low_high = data[(data['租金'] < rent_low) | (data['租金'] > rent_high)]
rental_low_high.to_csv(
    "./errorchecking/租金价格异常.csv", encoding="utf-8", index=False)

# 筛选出不在参考价格范围内的租金
rental_out_of_bounds = data[(data['租金'] < data['最低参考价'])
                            | (data['租金'] > data['最高参考价'])]
rental_out_of_bounds.to_csv("./errorchecking/租金超出上下界.csv",
                            encoding="utf-8", index=False)

# 计算面积的低和高分位数（5%和95%分位数）
area_low = data['面积'].quantile(0.05)
area_high = data['面积'].quantile(0.95)
area_low_high = data[(data['面积'] < area_low) | (data['面积'] > area_high)]
area_low_high.to_csv(
    "./errorchecking/面积异常.csv", encoding="utf-8", index=False)

# 按街道和小区分组，并提取最低和最高参考价格
price_summary = data.groupby(['街道', '小区'])[['最低参考价', '最高参考价']].agg([
    'min', 'max']).reset_index()

# 扁平化列标题
price_summary.columns = ['街道', '小区', '最低参考价_min',
                         '最低参考价_max', '最高参考价_min', '最高参考价_max']

# 保存到CSV文件
price_summary.to_csv("./errorchecking/小区更新区间.csv", index=False)

In [9]:
# 计算租金和面积的分位数
rent_low = data['租金'].quantile(0.05)
rent_high = data['租金'].quantile(0.95)
area_low = data['面积'].quantile(0.05)
area_high = data['面积'].quantile(0.95)

# 合并所有筛选条件
filtered_data = data[(data['租金'] >= rent_low) & (data['租金'] <= rent_high) &
                     (data['租金'] >= data['最低参考价']) & (data['租金'] <= data['最高参考价']) &
                     (data['面积'] >= area_low) & (data['面积'] <= area_high)]
filtered_data.to_csv('./output/dataforvisualization.csv', index=False)

In [10]:
# 根据描述选择要保留的列索引或名称列表
columns_to_keep = [
    '房源编号', '出租方式', '街道', '小区', '租金', '卧室数量', '客厅数量', '面积', '朝向',
    '装修情况', '关注人数', '近30天带看', '发布时间',
    '地铁直线距离', '公交直线距离', '地铁步行距离', '公交步行距离', '地铁步行时长', '公交步行时长',
    '1km内办公室', '1km内医院', '1km内健身房', '绿化率',
    '停车',
    '便利店', '综合超市', '商场', '药店', '银行', '小区物业费（元/平米·月）',
    '（最近医院）是否是三甲医院', '洗衣机', '冰箱', '电视', '空调', '热水器', '天然气', '暖气', '床', '网络', '衣柜',
    '附近餐厅数', '附近医院数'
]

# 选择保留的列
selected_data = filtered_data[columns_to_keep]

selected_data.to_csv('./output/filtered_data.csv', index=False)